In [1]:
# Load Libraries - Make sure to run this cell!
import pandas as pd
import numpy as np
import re, os
from string import printable
from sklearn import model_selection

#import gensim
import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model
from keras import regularizers
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten, Concatenate
from keras.layers import Input, ELU, LSTM, Embedding, Convolution2D, MaxPooling2D, \
BatchNormalization, Convolution1D, MaxPooling1D
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam, RMSprop
import np_utils
from keras import initializers
from keras import backend as K

from pathlib import Path
import json

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'tensorflow'


<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />

## Featureless Deep Learning

This notebook shows three commonly used neural network architectures to detect malicious URLs using **featureless Deep Learning**. [Keras](https://keras.io/) is used as high-level API for [tensorflow](https://www.tensorflow.org/) backend). 

Please refer to the slides (```Module 9 Featureless Deep Learning```) for additional info!


## Preprocess raw URLs

In [ ]:
## Load data URL

DATA_HOME = '../data/'
df = pd.read_csv(DATA_HOME + 'url_data_mega_deep_learning.csv')
df.sample(n=25).head(25) 

In [ ]:
# Initial Data Preparation URL

# Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer 

# Step 2: Cut URL string at max_len or pad with zeros if shorter
max_len=75
 
# Step 3: Extract labels form df to numpy array



In [ ]:
# Simple Cross-Validation: Split the data set into training and test data


In [ ]:
# GENERAL get layer dimensions for any model!
def print_layers_dims(model):
    l_layers = model.layers
    # Note None is ALWAYS batch_size
    for i in range(len(l_layers)):
        print(l_layers[i])
        print('Input Shape: ', l_layers[i].input_shape, 'Output Shape: ', l_layers[i].output_shape)

# GENERAL save model to disk function!
def save_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    #have h5py installed
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = model.to_json()
    with open(fileModelJSON,'w' ) as f:
        json.dump(json_string, f)
    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    model.save_weights(fileWeights)
    

# GENERAL load model from disk function!
def load_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    with open(fileModelJSON, 'r') as f:
         model_json = json.load(f)
         model = model_from_json(model_json)
    
    model.load_weights(fileWeights)
    return model

## Architecture 1 - Simple LSTM

In [ ]:
## Deep Learning model Definition --- A --- (Simple LSTM)


def simple_lstm(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len)(main_input) 
    
    # LSTM layer
    lstm = LSTM(lstm_output_size)(emb)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model(inputs=[main_input], outputs=[output])
    adam = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Fit model and Cross-Validation, ARCHITECTURE 1 SIMPLE LSTM
nb_epoch = 3
batch_size = 32

# Your code here.... 

## Architecture 2 - 1D Convolution and LSTM

In [ ]:
## Deep Learning model Definition --- B --- (1D Convolution and LSTM)

def lstm_conv(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                activity_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    # Conv layer
    conv = Convolution1D(filters=256, \
                     kernel_size=5)(emb)
    conv = ELU()(conv)

    conv = MaxPooling1D(pool_size=4)(conv)
    #conv = BatchNormalization(mode=0)(conv)
    conv = Dropout(0.5)(conv)

    # LSTM layer
    lstm = LSTM(lstm_output_size)(conv)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model([main_input], [output])
    adam = tf.keras.optimizers.legacy.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Fit model and Cross-Validation, ARCHITECTURE 2 CONV + LSTM
nb_epoch = 5
batch_size = 32

# Your code here...

## Architecture 3 - 1D Convolutions and Fully Connected Layers

In [ ]:
## Deep Learning model Definition --- C --- (1D Convolutions and Fully Connected Layers)

def conv_fully(max_len=75, emb_dim=32, max_vocab_len=100, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                activity_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    
    def sum_1d(X):
        return K.sum(X, axis=1)
    
    def get_conv_layer(emb, filter_length=5, nb_filter=256):
        # Conv layer
        conv = Convolution1D(kernel_size=filter_length, filters=nb_filter)(emb)
        conv = ELU()(conv)

        conv = Lambda(sum_1d, output_shape=(nb_filter,))(conv)
        #conv = BatchNormalization(mode=0)(conv)
        conv = Dropout(0.5)(conv)
        return conv
        
    # Multiple Conv Layers
    
    # calling custom conv function from above
    conv1 = get_conv_layer(emb, filter_length=2, nb_filter=256)
    conv2 = get_conv_layer(emb, filter_length=3, nb_filter=256)
    conv3 = get_conv_layer(emb, filter_length=4, nb_filter=256)
    conv4 = get_conv_layer(emb, filter_length=5, nb_filter=256)

    # Fully Connected Layers
    merged = tf.keras.layers.Concatenate()([conv1,conv2,conv3,conv4])

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization()(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(1024)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization()(hidden2)
    hidden2 = Dropout(0.5)(hidden2)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(hidden2)

    # Compile model and define optimizer
    model = Model([main_input], [output])
    adam = tf.keras.optimizers.legacy.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Fit model and Cross-Validation, ARCHITECTURE 3 CONV + FULLY CONNECTED
nb_epoch = 5
batch_size = 32

# Your code here ...

In [ ]:
# get probabilities of target predictions
target_proba = model.predict(X_test, batch_size=1)
target_proba[0:10]

## Making a new prediction

In [ ]:
test_url_mal = "naureen.net/etisalat.ae/index2.php"
test_url_benign = "sixt.com/php/reservation?language=en_US"

url = test_url_benign

In [ ]:
# Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer 
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable]]

# Step 2: Cut URL string at max_len or pad with zeros if shorter
max_len=75
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)

In [ ]:
target_proba = model.predict(X, batch_size=5)
def print_result(proba):
    if proba > 0.5:
        return "malicious"
    else:
        return "benign"
print("Test URL:", url, "is", print_result(target_proba[0]))